In [69]:
import pandas as pd
import numpy as np

# load train
data = pd.read_csv('data/train.csv')

# data aggregation by day_block
X = pd.DataFrame(data.groupby(['date_block_num','shop_id', 'item_id'])['item_cnt_day'].sum()).reset_index()

# load test
X_test = pd.read_csv('data/test.csv')

In [70]:
# for each row in test.csv file, do the following to obtain nearest value

# find the nearest month
ref_days = pd.merge(X_test,X,how='left',
         on=['shop_id', 'item_id'], suffixes=['_test','_train']).groupby(['shop_id', 'item_id'])['date_block_num'].max().reset_index()

# query sales of nearest sales
refs = pd.merge(ref_days, X,
                how='left',
                on=['shop_id', 'item_id', 'date_block_num'],)[['shop_id', 'item_id', 'item_cnt_day']]

# fill NAs using mean of all predictions, modify accordingly using your own algorithm
refs.loc[refs.item_cnt_day.isnull(), 'item_cnt_day'] = refs.item_cnt_day.mean()

In [65]:
# output to csv 
preds = pd.merge(X_test, refs, how='left', on=['shop_id', 'item_id']).rename(columns={'item_cnt_day':'item_cnt_month'})
preds.to_csv('predictions.csv', index_label='ID', columns=['item_cnt_month'])